In [1]:
import torch
from classes import FullFileForTestings  # Import dataset class and model
from classes import SpeedEstimatorRNNModified, SpeedEstimatorLSTMModified, SpeedEstimatorGRUModified
import os
import pandas as pd

In [2]:
#Loading in the hyperparameters
hyperparams_loc = "../2_trained_models/GRU/trained_models/i7/it_4_norm/hyperparams_GRU_it_4.csv"
hyperparams_df = pd.read_csv(hyperparams_loc, delimiter=";")

#Reading in the num of models and model locations
model_folder_loc = "../2_trained_models/GRU/trained_models/i7/it_4_norm/state_models/lon/"
pt_files = [f for f in os.listdir(model_folder_loc) if f.endswith(".pt")]

num_of_models = len(pt_files)

column_names = [None] * (num_of_models + 3)
for i in range(num_of_models):
    if i <= num_of_models:
        column_names[i + 1] = pt_files[i]

column_names[0] = 'time'
column_names[num_of_models + 1] = 'veh_u'
column_names[num_of_models + 2] = 'veh_v'

#Creating the dataframe for the storing the results
results_df = pd.DataFrame(columns = column_names)

#Loading in the reference speeds
validation_data_loc = "../1_data/i7/it_1/it_1_100_norm/3_validation/"
validation_files = [f for f in os.listdir(validation_data_loc) if f.endswith(".csv")]

csv_save_loc = "../2_trained_models/GRU/trained_models/i7/it_4_norm/results/"

In [3]:
for i in range(len(validation_files)):
    validation_df = pd.read_csv(validation_data_loc + validation_files[i], delimiter=",")

    results_df['time'] = validation_df['Time']
    results_df['veh_u'] = validation_df['veh_u']
    results_df['veh_v'] = validation_df['veh_v']

    #The test dataset is prepared for to be fed in to the model
    validation_dataset = FullFileForTestings(validation_data_loc + validation_files[i])
    features, actual_speeds = validation_dataset.get_full_data()

    for j in range(num_of_models):

        #Loading in the hyperparameters
        input_size = int(hyperparams_df["input_size"][j])
        hidden_size = int(hyperparams_df["hidden_size"][j])
        num_layers = int(hyperparams_df["num_of_layers"][j])
        output_size = int(hyperparams_df["output_size"][j])

        # Pass all data to the model directly
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        features = features.unsqueeze(0)  # Add batch dimension
        features = features.to(device)

        # Define and load the model
        model = SpeedEstimatorLSTMModified(input_size, hidden_size, num_layers, output_size)  # Instantiate your LSTM model
        checkpoint = torch.load(model_folder_loc + pt_files[j], map_location=device)

        model.load_state_dict(checkpoint['model_state_dict'])  # Load the trained weights
        model = model.to(device)  # Move model to the appropriate device
        model.eval()  # Set model to eval model

        # Inference
        with torch.no_grad():
            predictions = model(features)
            predicted_speeds = predictions.cpu().numpy().flatten()

        if output_size == 1:
            u = [0] * len(predicted_speeds)
            u = predicted_speeds

            results_df[pt_files[i]] = u
        else:
            length = len(predicted_speeds)/2
            u = [0]*int(length)
            v = [0]*int(length)

            for k in range(len(predicted_speeds)):
                if k % 2 == 0:
                    u[int(k/2)] = predicted_speeds[k]
                else:
                    v[int((k-1)/2)] = predicted_speeds[k]

    results_df.to_csv(csv_save_loc + validation_files[i], index=False)
    results_df = results_df.iloc[0:0]

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL numpy._core.multiarray.scalar was not an allowed global by default. Please use `torch.serialization.add_safe_globals([scalar])` or the `torch.serialization.safe_globals([scalar])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with weig

In [ ]:
for j in range(len(validation_files)):

    validation_df = pd.read_csv(validation_data_loc + validation_files[j], delimiter=",")

    results_df['time'] = validation_df['Time']
    results_df['veh_u'] = validation_df['veh_u']
    results_df['veh_v'] = validation_df['veh_v']

    #The test dataset is prepared for to be fed in to the model
    validation_dataset = FullFileForTestings(validation_data_loc + validation_files[j])
    features, actual_speeds = validation_dataset.get_full_data()

    for i in range(num_of_models):

        #Loading in the hyperparameters
        input_size = int(hyperparams_df["input_size"][i])
        hidden_size = int(hyperparams_df["hidden_size"][i])
        num_layers = int(hyperparams_df["num_of_layers"][i])
        output_size = int(hyperparams_df["output_size"][i])

        #Define and load the model
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Prepare features for inference
        features = features.unsqueeze(0).to(device)  # Add batch dimension

        model_loc = model_folder_loc + pt_files[i]
        checkpoint = torch.load(model_loc, map_location=device, weights_only=False)

        model = SpeedEstimatorGRUModified(input_size, hidden_size, num_layers, output_size)

        model.load_state_dict(checkpoint['model_state_dict'])
        model = model.to(device)
        model.eval()


         # Inference
        with torch.no_grad():
            predictions = model(features)
            predicted_speeds = predictions.cpu().numpy().flatten()

        if output_size == 1:
            results_df[pt_files[i]] = predicted_speeds
        #Itt le kell majd kezelni azt, amikor két kimenet van

    results_df.to_csv(csv_save_loc + validation_files[j], index=False)
    results_df = results_df.iloc[0:0]


In [5]:
for i in range(num_models):
    # Set paths and parameters for testing
    model_path = model_location + str(i) + '.pt'  # Replace with the path to your saved model

    for j in range(len(num_test_data_path)):
        test_csv_path = num_test_data_path[j]
        # Prepare the test dataset
        test_dataset = FullFileForTestings(test_csv_path)
        features, actual_speeds = test_dataset.get_full_data()

        # Pass all data to the model directly
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        features = features.unsqueeze(0)  # Add batch dimension
        features = features.to(device)

        # Define and load the model
        model = SpeedEstimatorLSTMModified(input_size, hidden_size[i], num_layers[i], output_size)  # Instantiate your LSTM model
        checkpoint = torch.load(model_path, map_location=device)

        model.load_state_dict(checkpoint['model_state_dict'])  # Load the trained weights
        model = model.to(device)  # Move model to the appropriate device
        model.eval()  # Set model to eval model

        # Inference
        with torch.no_grad():
            predictions = model(features)
            predicted_speeds = predictions.cpu().numpy().flatten()

        length = len(predicted_speeds)/2
        u = [0]*int(length)
        v = [0]*int(length)

        for k in range(len(predicted_speeds)):
            if k % 2 == 0:
                u[int(k/2)] = predicted_speeds[k]
            else:
                v[int((k-1)/2)] = predicted_speeds[k]



------------------------------------------------------------------------------------------------
debugging

In [ ]:
import matplotlib.pyplot as plt

#Plotting
# longitudinal plots
plt.figure(figsize=(10, 6))
plt.plot(actual_speeds[:, 0], label="Actual Speed")
plt.plot(u, label="Estimated Speed", linestyle="dashed")
plt.legend()
plt.title("Comparison of Actual vs. Predicted Longitudinal Speed (" + test_data[j] + ")")
plt.xlabel("Time Step [s]")
plt.ylabel("Speed [m/s]")
plt.grid()
plt.savefig(plot_location + str(i) + "/u/model_LSTM_" + str(i) + "_" + test_data[j] + "_act_vs_predicted_u.png", dpi=300,)
plt.close()

plt.figure(figsize=(10, 6))
plt.plot(actual_speeds[:, 0].numpy() - u, label="Difference")
plt.legend()
plt.title("Difference of Actual vs. Predicted Longitudinal Speed (" + test_data[j] + ")")
plt.xlabel("Time Step [s]")
plt.ylabel("Speed difference [m/s]")
plt.grid()
plt.savefig(plot_location + str(i) + "/u/model_LSTM_" + str(i) + "_" + test_data[j] + "_act_vs_predicted_u_diff.png", dpi=300)
plt.close()

# lateral plots
plt.figure(figsize=(10, 6))
plt.plot(actual_speeds[:, 1], label="Actual Speed")
plt.plot(v, label="Estimated Speed", linestyle="dashed")
plt.legend()
plt.title("Comparison of Actual vs. Predicted Lateral Speed (" + test_data[j] + ")")
plt.xlabel("Time Step [s]")
plt.ylabel("Speed [m/s]")
plt.grid()
plt.savefig(plot_location + str(i) + "/v/model_LSTM_" + str(i) + "_" + test_data[j] + "_act_vs_predicted_v.png", dpi=300,)
plt.close()

plt.figure(figsize=(10, 6))
plt.plot(actual_speeds[:, 1].numpy() - v, label="Difference")
plt.legend()
plt.title("Difference of Actual vs. Predicted Lateral Speed (" + test_data[j] + ")")
plt.xlabel("Time Step [s]")
plt.ylabel("Speed difference [m/s]")
plt.grid()
plt.savefig(plot_location + str(i) + "/v/model_LSTM_" + str(i) + "_" + test_data[j] + "_act_vs_predicted_v_diff.png", dpi=300)
plt.close()